In [1]:
# Loading in libraries
import pandas as pd
import matplotlib.pyplot as plt
import spatial_efd
import math 
import signac
import numpy as np
import os.path
import os
import torch
import gpytorch
from matplotlib import pyplot as plt
from sklearn.preprocessing import StandardScaler
import similaritymeasures
#from smt.sampling_methods import LHS
# Importing helper libraries for bayesian optimization
from dependencies.data_preprocessing_class import dataPreprocessing
from dependencies.gaussian_process_regression_class import gaussianProcessRegression
from dependencies.acquisition_functions_class import acqisitionFunctions
from dependencies.geometry_writer import geometryWriter
from dependencies.feature_extractor_4 import FeatureExtractor


In [2]:
# Checking if data exists
doesDataFileExist = os.path.isfile("master_feature_output.npy")
# Loading datafiles if they exist
# Else fetching and preparing data from signac workspace
if doesDataFileExist == True:
	master_parameter_input_n = np.load('master_parameter_input_n.npy', )
	master_feature_output = np.load('master_feature_output.npy', )

In [3]:
print(np.shape(master_feature_output))

(150, 80)


### Preparing input data for the GPR model

In [4]:
# Loading in the data processing class
dataPreprocess  = dataPreprocessing(master_parameter_input_n, master_feature_output, 150)
# Converting the input parameters to logscale
master_parameter_input_log = dataPreprocess.inputLogTransform()
# Selecting the parameters that were sampled in the latin hypercube sampling
num_parameters_LHS = 7
LHS_parameter_index = [1, 4, 7, 17, 18, 19, 33]
# Calling in the function to separate out the desired parameters
data_x = dataPreprocess.inputParameterSelection(num_parameters_LHS, LHS_parameter_index, master_parameter_input_log)
# Storing mean and standard deviation of input training data for later use
data_x_mean = np.mean(data_x, axis=0)
data_x_variance = np.std(data_x, axis=0)
# Normalizing data
data_x = StandardScaler().fit_transform(data_x)
# Estimating bounds for input transformed parameters
max_data_x = np.amax(data_x, axis=0) 
min_data_x = np.amin(data_x, axis=0) 
print(max_data_x)
print(min_data_x)


[1.7149018 1.7149018 1.7149018 1.7149018 1.7149018 1.7149018 1.7149018]
[-1.7149018 -1.7149018 -1.7149018 -1.7149018 -1.7149018 -1.7149018
 -1.7149018]


/home/nkumar4/Desktop/bayesian_optimization_frechet_2/dependencies/data_preprocessing_class.py:42: RuntimeWarning: divide by zero encountered in log
  master_parameter_input = np.log(self.master_parameter_input_n)


### Loading dataset for parameter estimation: Used for estimating fraehet distance and output for the GPR model

In [5]:
"""
Source for the shape for which parameter estimation has to be carried out
0: Shape is generated from the experimental image data
1: Shape has been generated rom the 
"""

target_shape_source = 1

if target_shape_source == 0:
    # Reading the text file containing coordinates from exterior contour of the wing disc cross section
    geometry_data = 'input_data/vertices_target.txt' 
    if type(geometry_data) is str:
        # Checking if the file containing vertices coordinates are empty
        if os.stat(geometry_data).st_size != 0:
            a1 = []
            a2 = []

            with open(geometry_data) as f:
                # next(f)
                for line in f:
                    data = line.split()
                    a1.append(float(data[0]))
                    a2.append(float(data[1]))


        else:
            a1 = 0
            a2 = 0

    vpos_x_exp = a1
    vpos_y_exp = a2

    coeffs = spatial_efd.CalculateEFD(vpos_x_exp, vpos_y_exp, 20)
    # Normalizing the coefficients against rotation and size
    coeffs_exp, rotation = spatial_efd.normalize_efd(coeffs, size_invariant=True)
    
elif target_shape_source == 1:
    # Reading the vertices output file from a sample SE simulation output with known parameters
    fe_exp = FeatureExtractor('input_data/vertices_target_SE.txt', 'log_edges.xlsx')
    # Extracting the efd coefficients
    coeffs_exp = fe_exp.tissue_efd_coeff(20)

### Preparing output dataset for GPR modelingf by defining frechet distance

In [6]:
# Reverse EFD for plotting the normalized tissue shape



"""
xt_exp, yt_exp = spatial_efd.inverse_transform(coeffs_exp, harmonic=20)
exp_data = np.zeros((300,2))
exp_data[:,0] = xt_exp
exp_data[:,1] = yt_exp

error_simulation_experimental_data = np.zeros(150)
for i in range(150):
    temp = master_feature_output[i,:]
    temp2 = np.reshape(temp, (20,4))
    xt, yt = spatial_efd.inverse_transform(temp2, harmonic=20)
    sim_data = np.zeros((300,2))
    sim_data[:,0] = xt
    sim_data[:,1] = yt
    error_simulation_experimental_data[i] = similaritymeasures.frechet_dist(exp_data,sim_data)
    
data_y = (np.reshape(error_simulation_experimental_data, (150,1)))*(-1)
print(np.shape(data_y))
np.save('data_y.npy', data_y)
"""

data_y = np.load('data_y.npy', )

### GPR modeling

In [7]:
class ExactGPModel(gpytorch.models.ExactGP):
	def __init__(self, train_x, train_y, likelihood):
		super(ExactGPModel, self).__init__(train_x, train_y, likelihood)
		self.mean_module = gpytorch.means.ConstantMean()
		self.covar_module = gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel(ard_num_dims=7))
		# self.white_noise = gpytorch.kernels.ScaleKernel(gpytorch.kernels.WhiteNoiseKernel())
		
	def forward(self, x):
		mean_x = self.mean_module(x)
		covar_x = self.covar_module(x)
		return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

split_size = 149
num_samples = 150
train_x = data_x[:split_size,:]
train_y = data_y[:split_size,:]
test_x = data_x[split_size:num_samples,:]
test_y = data_y[split_size:num_samples,:]

gpr = gaussianProcessRegression(train_x, train_y, test_x, test_y)
model, likelihood, train_x_t, train_y_t, test_x_t, test_y_pc = gpr.GP_model_definition(ExactGPModel,0,1)
model, likelihood, lengthscale_hyperparameters = gpr.GP_training(train_x_t, train_y_t,model,likelihood,1, 5000)

Kernel lengthscale has been initialized
Using Adam Optimizer
Iter 1/5000 - Loss: 1.084
Iter 2/5000 - Loss: 1.033
Iter 3/5000 - Loss: 0.985
Iter 4/5000 - Loss: 0.938
Iter 5/5000 - Loss: 0.891
Iter 6/5000 - Loss: 0.843
Iter 7/5000 - Loss: 0.794
Iter 8/5000 - Loss: 0.742
Iter 9/5000 - Loss: 0.690
Iter 10/5000 - Loss: 0.637
Iter 11/5000 - Loss: 0.583
Iter 12/5000 - Loss: 0.529
Iter 13/5000 - Loss: 0.476
Iter 14/5000 - Loss: 0.422
Iter 15/5000 - Loss: 0.369
Iter 16/5000 - Loss: 0.317
Iter 17/5000 - Loss: 0.265
Iter 18/5000 - Loss: 0.213
Iter 19/5000 - Loss: 0.162
Iter 20/5000 - Loss: 0.111
Iter 21/5000 - Loss: 0.061
Iter 22/5000 - Loss: 0.012
Iter 23/5000 - Loss: -0.037
Iter 24/5000 - Loss: -0.085
Iter 25/5000 - Loss: -0.132
Iter 26/5000 - Loss: -0.178
Iter 27/5000 - Loss: -0.222
Iter 28/5000 - Loss: -0.265
Iter 29/5000 - Loss: -0.307
Iter 30/5000 - Loss: -0.346
Iter 31/5000 - Loss: -0.384
Iter 32/5000 - Loss: -0.420
Iter 33/5000 - Loss: -0.454
Iter 34/5000 - Loss: -0.486
Iter 35/5000 - Los

Iter 322/5000 - Loss: -0.879
Iter 323/5000 - Loss: -0.879
Iter 324/5000 - Loss: -0.879
Iter 325/5000 - Loss: -0.879
Iter 326/5000 - Loss: -0.879
Iter 327/5000 - Loss: -0.879
Iter 328/5000 - Loss: -0.879
Iter 329/5000 - Loss: -0.879
Iter 330/5000 - Loss: -0.879
Iter 331/5000 - Loss: -0.879
Iter 332/5000 - Loss: -0.879
Iter 333/5000 - Loss: -0.879
Iter 334/5000 - Loss: -0.879
Iter 335/5000 - Loss: -0.879
Iter 336/5000 - Loss: -0.879
Iter 337/5000 - Loss: -0.879
Iter 338/5000 - Loss: -0.879
Iter 339/5000 - Loss: -0.879
Iter 340/5000 - Loss: -0.879
Iter 341/5000 - Loss: -0.879
Iter 342/5000 - Loss: -0.879
Iter 343/5000 - Loss: -0.879
Iter 344/5000 - Loss: -0.879
Iter 345/5000 - Loss: -0.879
Iter 346/5000 - Loss: -0.879
Iter 347/5000 - Loss: -0.879
Iter 348/5000 - Loss: -0.879
Iter 349/5000 - Loss: -0.879
Iter 350/5000 - Loss: -0.879
Iter 351/5000 - Loss: -0.879
Iter 352/5000 - Loss: -0.879
Iter 353/5000 - Loss: -0.879
Iter 354/5000 - Loss: -0.879
Iter 355/5000 - Loss: -0.879
Iter 356/5000 

Iter 626/5000 - Loss: -0.879
Iter 627/5000 - Loss: -0.879
Iter 628/5000 - Loss: -0.879
Iter 629/5000 - Loss: -0.879
Iter 630/5000 - Loss: -0.878
Iter 631/5000 - Loss: -0.878
Iter 632/5000 - Loss: -0.879
Iter 633/5000 - Loss: -0.879
Iter 634/5000 - Loss: -0.879
Iter 635/5000 - Loss: -0.879
Iter 636/5000 - Loss: -0.879
Iter 637/5000 - Loss: -0.879
Iter 638/5000 - Loss: -0.879
Iter 639/5000 - Loss: -0.879
Iter 640/5000 - Loss: -0.879
Iter 641/5000 - Loss: -0.879
Iter 642/5000 - Loss: -0.879
Iter 643/5000 - Loss: -0.879
Iter 644/5000 - Loss: -0.879
Iter 645/5000 - Loss: -0.879
Iter 646/5000 - Loss: -0.879
Iter 647/5000 - Loss: -0.879
Iter 648/5000 - Loss: -0.879
Iter 649/5000 - Loss: -0.879
Iter 650/5000 - Loss: -0.879
Iter 651/5000 - Loss: -0.879
Iter 652/5000 - Loss: -0.879
Iter 653/5000 - Loss: -0.879
Iter 654/5000 - Loss: -0.879
Iter 655/5000 - Loss: -0.879
Iter 656/5000 - Loss: -0.879
Iter 657/5000 - Loss: -0.879
Iter 658/5000 - Loss: -0.879
Iter 659/5000 - Loss: -0.879
Iter 660/5000 

Iter 931/5000 - Loss: -0.879
Iter 932/5000 - Loss: -0.879
Iter 933/5000 - Loss: -0.879
Iter 934/5000 - Loss: -0.879
Iter 935/5000 - Loss: -0.879
Iter 936/5000 - Loss: -0.879
Iter 937/5000 - Loss: -0.879
Iter 938/5000 - Loss: -0.879
Iter 939/5000 - Loss: -0.879
Iter 940/5000 - Loss: -0.879
Iter 941/5000 - Loss: -0.879
Iter 942/5000 - Loss: -0.879
Iter 943/5000 - Loss: -0.879
Iter 944/5000 - Loss: -0.879
Iter 945/5000 - Loss: -0.879
Iter 946/5000 - Loss: -0.879
Iter 947/5000 - Loss: -0.879
Iter 948/5000 - Loss: -0.879
Iter 949/5000 - Loss: -0.879
Iter 950/5000 - Loss: -0.879
Iter 951/5000 - Loss: -0.879
Iter 952/5000 - Loss: -0.879
Iter 953/5000 - Loss: -0.879
Iter 954/5000 - Loss: -0.879
Iter 955/5000 - Loss: -0.879
Iter 956/5000 - Loss: -0.879
Iter 957/5000 - Loss: -0.879
Iter 958/5000 - Loss: -0.879
Iter 959/5000 - Loss: -0.879
Iter 960/5000 - Loss: -0.879
Iter 961/5000 - Loss: -0.879
Iter 962/5000 - Loss: -0.879
Iter 963/5000 - Loss: -0.879
Iter 964/5000 - Loss: -0.879
Iter 965/5000 

Iter 1222/5000 - Loss: -0.879
Iter 1223/5000 - Loss: -0.879
Iter 1224/5000 - Loss: -0.879
Iter 1225/5000 - Loss: -0.879
Iter 1226/5000 - Loss: -0.879
Iter 1227/5000 - Loss: -0.879
Iter 1228/5000 - Loss: -0.879
Iter 1229/5000 - Loss: -0.879
Iter 1230/5000 - Loss: -0.879
Iter 1231/5000 - Loss: -0.879
Iter 1232/5000 - Loss: -0.879
Iter 1233/5000 - Loss: -0.879
Iter 1234/5000 - Loss: -0.879
Iter 1235/5000 - Loss: -0.879
Iter 1236/5000 - Loss: -0.879
Iter 1237/5000 - Loss: -0.879
Iter 1238/5000 - Loss: -0.879
Iter 1239/5000 - Loss: -0.879
Iter 1240/5000 - Loss: -0.879
Iter 1241/5000 - Loss: -0.879
Iter 1242/5000 - Loss: -0.879
Iter 1243/5000 - Loss: -0.879
Iter 1244/5000 - Loss: -0.879
Iter 1245/5000 - Loss: -0.879
Iter 1246/5000 - Loss: -0.879
Iter 1247/5000 - Loss: -0.879
Iter 1248/5000 - Loss: -0.879
Iter 1249/5000 - Loss: -0.879
Iter 1250/5000 - Loss: -0.879
Iter 1251/5000 - Loss: -0.879
Iter 1252/5000 - Loss: -0.879
Iter 1253/5000 - Loss: -0.879
Iter 1254/5000 - Loss: -0.879
Iter 1255/

...

Iter 3563/5000 - Loss: -0.879
Iter 3564/5000 - Loss: -0.879
Iter 3565/5000 - Loss: -0.879
Iter 3566/5000 - Loss: -0.879
Iter 3567/5000 - Loss: -0.879
Iter 3568/5000 - Loss: -0.879
Iter 3569/5000 - Loss: -0.879
Iter 3570/5000 - Loss: -0.879
Iter 3571/5000 - Loss: -0.879
Iter 3572/5000 - Loss: -0.879
Iter 3573/5000 - Loss: -0.879
Iter 3574/5000 - Loss: -0.879
Iter 3575/5000 - Loss: -0.879
Iter 3576/5000 - Loss: -0.879
Iter 3577/5000 - Loss: -0.879
Iter 3578/5000 - Loss: -0.879
Iter 3579/5000 - Loss: -0.879
Iter 3580/5000 - Loss: -0.879
Iter 3581/5000 - Loss: -0.879
Iter 3582/5000 - Loss: -0.879
Iter 3583/5000 - Loss: -0.879
Iter 3584/5000 - Loss: -0.879
Iter 3585/5000 - Loss: -0.879
Iter 3586/5000 - Loss: -0.879
Iter 3587/5000 - Loss: -0.879
Iter 3588/5000 - Loss: -0.879
Iter 3589/5000 - Loss: -0.879
Iter 3590/5000 - Loss: -0.879
Iter 3591/5000 - Loss: -0.879
Iter 3592/5000 - Loss: -0.879
Iter 3593/5000 - Loss: -0.879
Iter 3594/5000 - Loss: -0.879
Iter 3595/5000 - Loss: -0.879
Iter 3596/

Iter 3876/5000 - Loss: -0.879
Iter 3877/5000 - Loss: -0.879
Iter 3878/5000 - Loss: -0.879
Iter 3879/5000 - Loss: -0.879
Iter 3880/5000 - Loss: -0.879
Iter 3881/5000 - Loss: -0.879
Iter 3882/5000 - Loss: -0.879
Iter 3883/5000 - Loss: -0.879
Iter 3884/5000 - Loss: -0.879
Iter 3885/5000 - Loss: -0.879
Iter 3886/5000 - Loss: -0.879
Iter 3887/5000 - Loss: -0.879
Iter 3888/5000 - Loss: -0.879
Iter 3889/5000 - Loss: -0.879
Iter 3890/5000 - Loss: -0.879
Iter 3891/5000 - Loss: -0.879
Iter 3892/5000 - Loss: -0.879
Iter 3893/5000 - Loss: -0.879
Iter 3894/5000 - Loss: -0.879
Iter 3895/5000 - Loss: -0.879
Iter 3896/5000 - Loss: -0.879
Iter 3897/5000 - Loss: -0.879
Iter 3898/5000 - Loss: -0.879
Iter 3899/5000 - Loss: -0.879
Iter 3900/5000 - Loss: -0.879
Iter 3901/5000 - Loss: -0.879
Iter 3902/5000 - Loss: -0.879
Iter 3903/5000 - Loss: -0.879
Iter 3904/5000 - Loss: -0.879
Iter 3905/5000 - Loss: -0.879
Iter 3906/5000 - Loss: -0.879
Iter 3907/5000 - Loss: -0.879
Iter 3908/5000 - Loss: -0.879
Iter 3909/

Iter 4187/5000 - Loss: -0.879
Iter 4188/5000 - Loss: -0.879
Iter 4189/5000 - Loss: -0.879
Iter 4190/5000 - Loss: -0.879
Iter 4191/5000 - Loss: -0.879
Iter 4192/5000 - Loss: -0.879
Iter 4193/5000 - Loss: -0.879
Iter 4194/5000 - Loss: -0.879
Iter 4195/5000 - Loss: -0.879
Iter 4196/5000 - Loss: -0.879
Iter 4197/5000 - Loss: -0.879
Iter 4198/5000 - Loss: -0.879
Iter 4199/5000 - Loss: -0.879
Iter 4200/5000 - Loss: -0.879
Iter 4201/5000 - Loss: -0.879
Iter 4202/5000 - Loss: -0.879
Iter 4203/5000 - Loss: -0.879
Iter 4204/5000 - Loss: -0.879
Iter 4205/5000 - Loss: -0.879
Iter 4206/5000 - Loss: -0.879
Iter 4207/5000 - Loss: -0.879
Iter 4208/5000 - Loss: -0.879
Iter 4209/5000 - Loss: -0.879
Iter 4210/5000 - Loss: -0.879
Iter 4211/5000 - Loss: -0.879
Iter 4212/5000 - Loss: -0.879
Iter 4213/5000 - Loss: -0.879
Iter 4214/5000 - Loss: -0.879
Iter 4215/5000 - Loss: -0.879
Iter 4216/5000 - Loss: -0.879
Iter 4217/5000 - Loss: -0.879
Iter 4218/5000 - Loss: -0.879
Iter 4219/5000 - Loss: -0.879
Iter 4220/

Iter 4466/5000 - Loss: -0.879
Iter 4467/5000 - Loss: -0.879
Iter 4468/5000 - Loss: -0.879
Iter 4469/5000 - Loss: -0.879
Iter 4470/5000 - Loss: -0.879
Iter 4471/5000 - Loss: -0.879
Iter 4472/5000 - Loss: -0.879
Iter 4473/5000 - Loss: -0.879
Iter 4474/5000 - Loss: -0.879
Iter 4475/5000 - Loss: -0.879
Iter 4476/5000 - Loss: -0.879
Iter 4477/5000 - Loss: -0.879
Iter 4478/5000 - Loss: -0.879
Iter 4479/5000 - Loss: -0.879
Iter 4480/5000 - Loss: -0.879
Iter 4481/5000 - Loss: -0.879
Iter 4482/5000 - Loss: -0.879
Iter 4483/5000 - Loss: -0.879
Iter 4484/5000 - Loss: -0.879
Iter 4485/5000 - Loss: -0.879
Iter 4486/5000 - Loss: -0.879
Iter 4487/5000 - Loss: -0.879
Iter 4488/5000 - Loss: -0.879
Iter 4489/5000 - Loss: -0.879
Iter 4490/5000 - Loss: -0.879
Iter 4491/5000 - Loss: -0.879
Iter 4492/5000 - Loss: -0.879
Iter 4493/5000 - Loss: -0.879
Iter 4494/5000 - Loss: -0.879
Iter 4495/5000 - Loss: -0.879
Iter 4496/5000 - Loss: -0.879
Iter 4497/5000 - Loss: -0.879
Iter 4498/5000 - Loss: -0.879
Iter 4499/

Iter 4748/5000 - Loss: -0.879
Iter 4749/5000 - Loss: -0.879
Iter 4750/5000 - Loss: -0.879
Iter 4751/5000 - Loss: -0.879
Iter 4752/5000 - Loss: -0.879
Iter 4753/5000 - Loss: -0.879
Iter 4754/5000 - Loss: -0.879
Iter 4755/5000 - Loss: -0.879
Iter 4756/5000 - Loss: -0.879
Iter 4757/5000 - Loss: -0.879
Iter 4758/5000 - Loss: -0.879
Iter 4759/5000 - Loss: -0.879
Iter 4760/5000 - Loss: -0.879
Iter 4761/5000 - Loss: -0.879
Iter 4762/5000 - Loss: -0.879
Iter 4763/5000 - Loss: -0.879
Iter 4764/5000 - Loss: -0.879
Iter 4765/5000 - Loss: -0.879
Iter 4766/5000 - Loss: -0.879
Iter 4767/5000 - Loss: -0.879
Iter 4768/5000 - Loss: -0.879
Iter 4769/5000 - Loss: -0.879
Iter 4770/5000 - Loss: -0.879
Iter 4771/5000 - Loss: -0.879
Iter 4772/5000 - Loss: -0.879
Iter 4773/5000 - Loss: -0.879
Iter 4774/5000 - Loss: -0.879
Iter 4775/5000 - Loss: -0.879
Iter 4776/5000 - Loss: -0.879
Iter 4777/5000 - Loss: -0.879
Iter 4778/5000 - Loss: -0.879
Iter 4779/5000 - Loss: -0.879
Iter 4780/5000 - Loss: -0.879
Iter 4781/